In [1]:
import os
import csv
import glob
import pandas as pd
from collections import defaultdict

import random
from scipy.stats import pearsonr
from sklearn import linear_model

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

### Performer lists

In [79]:
#getting a list of files in the performers folder

performers_folder = "/Users/emilija/Desktop/homepages/Performers"
dir_l = os.listdir(performers_folder)
for file in dir_l:
    if file.startswith("."):
        dir_l.remove(file)
print(dir_l)


['performerlist_bongacams.csv', 'performerlist_myfreecams.csv', 'performerlist_chaturbate.csv', 'performerlist_sample_1000.csv', 'performerlist_livejasmin.csv', 'performerlist_ebonycams.csv', 'performerlist_streamate.csv']


In [80]:
#choosing a random file and printing it's name 
test_file = random.choice(dir_l)
print(test_file)

#reading the chosen file into a pandas dataframe and opening the output
test_df = pd.read_csv(f"{performers_folder}/{test_file}")
test_df

performerlist_sample_1000.csv


,Unnamed: 0,name,times_online,weeks_online,calendar_weeks_online,counted,location,age,has_been_promoted,gender,is_couple,url,tags,rank_average,rank_total,viewers_average,viewers_total,minutes_online_average,minutes_online_total,minutes_online_last_time
0,7709,parisrose9,29,7,"44,45,46,47,48,49,50",203,"New York, United States",,False,female,False,https://en.chaturbate.com/parisrose9/,"#natural, #smoke,#lovense, #feet, #latino",5185,1052640,13,2706,208,6042,372
1,11689,ashley_stone2,60,11,"44,45,46,47,48,49,50,51,52,1,2",577,"Medellin, Colombia",,False,female,False,https://en.chaturbate.com/ashley_stone2/,"#latina,#blowjob, #smalltits, #ass, #young, #...",4085,2357134,4,2708,314,18857,78
2,26034,cataleya_prinss,15,8,"44,45,46,47,48,49,50,51",89,colombia.,20,False,female,False,https://en.chaturbate.com/cataleya_prinss/,"#squirt,#lovense, #cum, #18, #anal",4223,375892,2,208,168,2532,32
3,164324,nakedjoker98,1,1,52,2,Who knows,,False,male,False,https://en.chaturbate.com/nakedjoker98/,"#young, #solo, #gay, #jerking, #jerkoff",3303,6606,9,18,29,29,29
4,21745,tevagustarcock,25,11,"44,45,46,47,48,49,50,51,52,1,2",65,España,63,False,male,False,https://en.chaturbate.com/tevagustarcock/,NaN,3046,198041,29,1911,64,1604,43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,117533,alexey8777,5,2,"47,51",10,Russia,35,False,male,False,https://en.chaturbate.com/alexey8777/,NaN,5867,58675,21,214,78,394,28
996,88598,christie_monteiro,1,1,45,5,In your dreams,22,False,female,False,https://en.chaturbate.com/christie_monteiro/,"#18, #new, #daddy, #c2c, #natural",2237,11187,32,160,180,180,180
997,157324,camerryn,4,1,51,12,"Colorado, United States",,False,female,False,https://en.chaturbate.com/camerryn/,#lovense,113,1365,1239,14879,115,460,46
998,98483,ramme71,4,3,"45,50,52",8,My imagination,,False,male,False,https://en.chaturbate.com/ramme71/,NaN,5854,46837,2,16,22,90,19


### Looking for performer overlap between different platforms 
Assuming performers use the same screen names on different platforms

1. collect all unique performers
2. make a dictionary with unique performers and which platforms they appear on
3. make that into dataframe + csv file
4. make separate dataframes for overlap between main 5 platforms and between streamate+whitelabel
5. separate only performers that appear at least on 2 platforms and get additional metrics from those files (for now, average ranking per platform)


In [83]:
#making a list of unique performers throughout all platforms 
unique_performers = set()
performers = {}

#checks for each filename in the folder
for filename in dir_l:
    #print(filename) #prints the filename
    file=f"{performers_folder}/{filename}" #accesses the file with the file name 
    performers[filename] = set() #creates a set for unique performers for each file in a dictionary

    with open(file, "r") as f: 
        reader = csv.DictReader(f) #opens the file
        for row in reader: #reads every row of the csv
            performer = row["name"] #grabs the name 
            unique_performers.add(performer) # adds the performer to the overall unique performer set
            performers[filename].add(performer) #adds the performer to the dictionary of performers per each file 
            # looks like this: {filename:{emilija,pijus,solveiga,...}, filename2:{cassandra, jekaterina,...},...}

    print(f"{filename} file has {len(performers[filename])} unique performers")
print(f"in total there are {len(unique_performers)} unique performers in full dataset")

performerlist_bongacams.csv file has 31170 unique performers
performerlist_myfreecams.csv file has 20389 unique performers
performerlist_chaturbate.csv file has 190832 unique performers
performerlist_sample_1000.csv file has 1000 unique performers
performerlist_livejasmin.csv file has 38203 unique performers
performerlist_ebonycams.csv file has 30708 unique performers
performerlist_streamate.csv file has 30794 unique performers
in total there are 306182 unique performers in full dataset


In [23]:
#next, i am making a dictionary like this 
# overlap = {
#   cassandra: {filename1: True, filename2: False}, 
#   emilija: {filename1: True, filename2: True}, ...
# }
# then i use it to make a csv file like this:
# name (from all unique performers) | filename1  | filename2 | ...
# cassandra                         | True       | False     |...

overlap = defaultdict(dict) #making the overlap dictionary with all performer names as keys
#print(overlap)

for key in performers.keys(): #looping through all keys in the performers dictionary that stores performers per filename
    #print(key)
    for perf in unique_performers: #looping though all performers in the set of unique performers
        #print(perf)
        if perf in performers[key]: #checking if a unique performer is in the specific file
            overlap[perf][key] = 1
        else:
            overlap[perf][key] = 0

overlap_df = pd.DataFrame(overlap).transpose() #makes a dataframe and flips the rows and columns
overlap_df["performer"] = overlap_df.index
overlap_df = overlap_df.drop(columns=["performerlist_sample_1000.csv"])
overlap_df


,performerlist_bongacams.csv,performerlist_myfreecams.csv,performerlist_chaturbate.csv,performerlist_livejasmin.csv,performerlist_ebonycams.csv,performerlist_streamate.csv,performer
,0,0,0,1,0,0,
cataleyarios_,0,0,1,0,0,0,cataleyarios_
ssjrico,0,0,1,0,0,0,ssjrico
JessyBruks,0,0,0,1,0,0,JessyBruks
jay_bby,0,0,1,0,0,0,jay_bby
...,...,...,...,...,...,...,...
Maluhka325,1,0,0,0,0,0,Maluhka325
Victoria-mend,1,0,0,0,0,0,Victoria-mend
sluttywhore_ella,0,0,1,0,0,0,sluttywhore_ella
Nadiechka1,1,0,0,0,0,0,Nadiechka1


In [85]:
#looking at overlap between the five main platforms and separately the overlap between streamate and its whitelabel ebonycams

overlap_df["overlap_main"] = overlap_df["performerlist_bongacams.csv"] + overlap_df["performerlist_myfreecams.csv"] + overlap_df["performerlist_chaturbate.csv"] + overlap_df["performerlist_livejasmin.csv"] + overlap_df["performerlist_streamate.csv"]
overlap_df["whitelabel_overlap"] = overlap_df["performerlist_streamate.csv"] + overlap_df["performerlist_ebonycams.csv"]
overlap_df.to_csv("performer_overlap_all.csv")

#dividing into two new dataframes: 
overlap_main_df = overlap_df[overlap_df.overlap_main != 0] #one for looking at the main 5 platform overlap, so deleting all rows where overlap between them is 0
overlap_main_df.to_csv("main_five_overlap.csv")
overlap_wl_df = overlap_df[overlap_df.whitelabel_overlap != 0] #and one for whitelabel overlap so removing all rows with 0 overlap between streamate and ebony
overlap_wl_df.to_csv("whitelabel_overlap.csv")

overlap_df


,performerlist_bongacams.csv,performerlist_myfreecams.csv,performerlist_chaturbate.csv,performerlist_livejasmin.csv,performerlist_ebonycams.csv,performerlist_streamate.csv,performer,overlap_main,whitelabel_overlap
,0,0,0,1,0,0,,1,0
cataleyarios_,0,0,1,0,0,0,cataleyarios_,1,0
ssjrico,0,0,1,0,0,0,ssjrico,1,0
JessyBruks,0,0,0,1,0,0,JessyBruks,1,0
jay_bby,0,0,1,0,0,0,jay_bby,1,0
...,...,...,...,...,...,...,...,...,...
Maluhka325,1,0,0,0,0,0,Maluhka325,1,0
Victoria-mend,1,0,0,0,0,0,Victoria-mend,1,0
sluttywhore_ella,0,0,1,0,0,0,sluttywhore_ella,1,0
Nadiechka1,1,0,0,0,0,0,Nadiechka1,1,0


In [77]:
# now checking those performers who appear on more than one main platform: are they generally "successful"? 
# first i remove all performers that appear less that 2 times

overlap_main_df.loc[overlap_main_df["overlap_main"] >= 2, "busy_performer"] = "True"
overlap_main_df.loc[overlap_main_df["overlap_main"] < 2, "busy_performer"] = "False"

busy_perf_df = overlap_main_df.copy()
busy_perf_df.drop(busy_perf_df[busy_perf_df.busy_performer == "False"].index, inplace=True)
#if overlap_main_df.loc[overlap_main_df["overlap_main"] == "True"]:
#    busy_perf_df = 
busy_perf_df.to_csv("busy_performers.csv")
busy_perf_df


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,performerlist_bongacams.csv,performerlist_myfreecams.csv,performerlist_chaturbate.csv,performerlist_livejasmin.csv,performerlist_ebonycams.csv,performerlist_streamate.csv,performer,overlap_main,whitelabel_overlap,busy_performer
OliviaFalcon,0,0,0,1,1,1,OliviaFalcon,2,2,True
GlamyWife,1,1,0,0,0,0,GlamyWife,2,0,True
kristen230821,1,0,1,0,0,0,kristen230821,2,0,True
MartinaPiero,0,0,0,1,1,1,MartinaPiero,2,2,True
OctaviaFelicity,0,0,0,1,1,1,OctaviaFelicity,2,2,True
...,...,...,...,...,...,...,...,...,...,...
ZiaVyxen,0,1,0,1,1,1,ZiaVyxen,3,2,True
annamontrix,1,0,1,0,0,0,annamontrix,2,0,True
VictoriaBelmonte,0,0,0,1,1,1,VictoriaBelmonte,2,2,True
queenblack_cock,0,0,1,0,1,1,queenblack_cock,2,2,True


In [86]:
#can also make special labels for only private and only public show performers as well as "mixed", according on which platforms they co-stream
#should do the same with whitelabel vs streamate!

#this can be adapted to collect other overlapping data -- not only the average rank

#collecting all performers per platform if they appeared on more than one platform
bonga_busy = []
mfc_busy = []
chaturbate_busy = []
jasmin_busy = []
streamate_busy = []
"""
#adding new columns to main dataframe of busy performers, the values are empty and will be filled only when data exists
busy_perf_df["rank_average_bonga"] = None
busy_perf_df["rank_average_mfc"] = None
busy_perf_df["rank_average_chaturbate"] = None
busy_perf_df["rank_average_livejasmin"] = None
busy_perf_df["rank_average_streamate"] = None
""" 
#collecting performers into lists
with open("busy_performers.csv", "r") as file:
    reader = csv.DictReader(file)
    for row in reader:
        #print(row)
        if row["performerlist_bongacams.csv"] == "1":
            bonga_busy.append(row["performer"])
            #print("added bonga performer")
        
        if row["performerlist_myfreecams.csv"] == "1":
            mfc_busy.append(row["performer"])
            #print("added myfreecams performer")
        
        if row["performerlist_chaturbate.csv"] == "1":
            chaturbate_busy.append(row["performer"])
            #print("added chaturbate performer")
        
        if row["performerlist_livejasmin.csv"] == "1":
            jasmin_busy.append(row["performer"])
            #print("added livejasmin performer")
        
        if row["performerlist_streamate.csv"] == "1":
            streamate_busy.append(row["performer"])
            #print("added streamate performer")

#opening each performer list one by one, looking up the busy performers from each list and adding they average rank to the main dataframe
df = pd.read_csv(f"{performers_folder}/performerlist_bongacams.csv")
for perf in bonga_busy:
    rank = df.loc[df["name"] == perf, "rank_average"]
    #print(rank)
    busy_perf_df.at[perf, "rank_average_bonga"] = float(rank)

df = pd.read_csv(f"{performers_folder}/performerlist_myfreecams.csv")
for perf in mfc_busy:
    rank = df.loc[df["name"] == perf, "rank_average"]
    #print(rank)
    busy_perf_df.at[perf, "rank_average_mfc"] = float(rank)

df = pd.read_csv(f"{performers_folder}/performerlist_chaturbate.csv")
for perf in chaturbate_busy:
    rank = df.loc[df["name"] == perf, "rank_average"]
    #print(rank)
    busy_perf_df.at[perf, "rank_average_chaturbate"] = float(rank)

df = pd.read_csv(f"{performers_folder}/performerlist_livejasmin.csv")
for perf in jasmin_busy:
    rank = df.loc[df["name"] == perf, "rank_average"]
    #print(rank)
    busy_perf_df.at[perf, "rank_average_livejasmin"] = float(rank)

df = pd.read_csv(f"{performers_folder}/performerlist_streamate.csv")
for perf in streamate_busy:
    rank = df.loc[df["name"] == perf, "rank_average"]
    #print(rank)
    busy_perf_df.at[perf, "rank_average_streamate"] = float(rank)

busy_perf_df


,performerlist_bongacams.csv,performerlist_myfreecams.csv,performerlist_chaturbate.csv,performerlist_livejasmin.csv,performerlist_ebonycams.csv,performerlist_streamate.csv,performer,overlap_main,whitelabel_overlap,busy_performer,rank_average_bonga,rank_average_mfc,rank_average_chaturbate,rank_average_livejasmin,rank_average_streamate
OliviaFalcon,0,0,0,1,1,1,OliviaFalcon,2,2,True,NaN,NaN,NaN,675.244898,1468.740947
GlamyWife,1,1,0,0,0,0,GlamyWife,2,0,True,202.019608,1051.138158,NaN,NaN,NaN
kristen230821,1,0,1,0,0,0,kristen230821,2,0,True,1351.687500,NaN,4677.0,NaN,NaN
MartinaPiero,0,0,0,1,1,1,MartinaPiero,2,2,True,NaN,NaN,NaN,2158.441558,316.220930
OctaviaFelicity,0,0,0,1,1,1,OctaviaFelicity,2,2,True,NaN,NaN,NaN,1189.643564,402.065259
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZiaVyxen,0,1,0,1,1,1,ZiaVyxen,3,2,True,NaN,777.600000,NaN,1076.538462,1086.615385
annamontrix,1,0,1,0,0,0,annamontrix,2,0,True,145.235217,NaN,3504.0,NaN,NaN
VictoriaBelmonte,0,0,0,1,1,1,VictoriaBelmonte,2,2,True,NaN,NaN,NaN,497.780488,554.600000
queenblack_cock,0,0,1,0,1,1,queenblack_cock,2,2,True,NaN,NaN,3650.0,NaN,1687.443396


### Checking what data about performances each platform file has
Looking at the available data from different performer list files

In [87]:
#reading the files in the performers folder and taking their column names, adding them all to one dataframe for comparison

columns_full = []
headers_l = []

for file in dir_l:
    file_dir = f"{performers_folder}/{file}"
    df = pd.read_csv(file_dir)
    columns_l = df.columns.tolist()
    print(f"{file} has these columns: {columns_l}")
    columns_full.append(columns_l)
    headers_l.append(file)

columns_df = pd.DataFrame(columns_full).transpose()
columns_df.columns = headers_l
columns_df = columns_df.drop("performerlist_sample_1000.csv", 1)
columns_df.to_csv("performerlist_data_overview.csv")
columns_df

performerlist_bongacams.csv has these columns: ['name', 'times_online', 'weeks_online', 'calendar_weeks_online', 'counted', 'has_been_promoted', 'social_media', 'url', 'tags', 'rank_average', 'rank_total', 'viewers_average', 'viewers_total', 'minutes_online_average', 'minutes_online_total', 'time_online_last_time']
performerlist_myfreecams.csv has these columns: ['name', 'times_online', 'weeks_online', 'calendar_weeks_online', 'counted', 'had_silver_crown', 'had_golden_crown', 'times_private', 'rank_average', 'rank_total', 'viewers_average', 'viewers_total', 'minutes_online_average', 'minutes_online_total', 'time_online_last_time']
performerlist_chaturbate.csv has these columns: ['name', 'times_online', 'weeks_online', 'calendar_weeks_online', 'counted', 'location', 'age', 'has_been_promoted', 'gender', 'is_couple', 'url', 'tags', 'rank_average', 'rank_total', 'viewers_average', 'viewers_total', 'minutes_online_average', 'minutes_online_total', 'minutes_online_last_time']
performerlist

,performerlist_bongacams.csv,performerlist_myfreecams.csv,performerlist_chaturbate.csv,performerlist_livejasmin.csv,performerlist_ebonycams.csv,performerlist_streamate.csv
0,name,name,name,name,name,name
1,times_online,times_online,times_online,times_online,times_online,times_online
2,weeks_online,weeks_online,weeks_online,weeks_online,weeks_online,weeks_online
3,calendar_weeks_online,calendar_weeks_online,calendar_weeks_online,calendar_weeks_online,calendar_weeks_online,calendar_weeks_online
4,counted,counted,counted,counted,counted,counted
5,has_been_promoted,had_silver_crown,location,has_been_promoted,age,age
6,social_media,had_golden_crown,age,url,rating,rating
7,url,times_private,has_been_promoted,rank_average,url,url
8,tags,rank_average,gender,rank_total,rank_average,rank_average
9,rank_average,rank_total,is_couple,minutes_online_average,rank_total,rank_total


In [90]:
#calculate a correlation between rank average and viewers average, if possible (depending on the test_file chosen):
try:
    correlation1 = pearsonr(test_df["rank_average"],test_df["viewers_average"])
    print(f"the correlation between rank average and viewers average on {test_file} is {correlation1}")
except:
    print(f"the test_file chosen was {test_file}, it is missing data on viewers")


the correlation between rank average and viewers average on performerlist_sample_1000.csv is (-0.23277438063043931, 9.028100939059335e-14)


In [92]:
#calculate a correlation between rank average and minutes online average / total

correlation2 = pearsonr(test_df["rank_average"],test_df["minutes_online_average"])
print(f"the correlation between rank average and minutes online average on {test_file} is {correlation2}")
correlation3 = pearsonr(test_df["rank_average"],test_df["minutes_online_total"])
print(f"the correlation between rank average and minutes online sum on {test_file} is {correlation3}")

the correlation between rank average and minutes online average on performerlist_sample_1000.csv is (-0.15140305308100957, 1.5127210820890439e-06)
the correlation between rank average and minutes online sum on performerlist_sample_1000.csv is (-0.1088465612191679, 0.0005648076120562011)


In [97]:
#calculate multivariate linear regression for rank from viewers and minutes in shows

X = test_df[["minutes_online_average", "viewers_average"]]
y = test_df["rank_average"]

minutes = 200 #choose a time interval for minutes average
viewers = 3000 #choose a number of viewers average

regr = linear_model.LinearRegression()
regr.fit(X, y)
predicted_rank = regr.predict([[minutes, viewers]])

print(regr.coef_)
print(f"if a performer spends on average a {minutes} minutes online in a show and has on average {viewers} viewers, their average ranking will be:{predicted_rank}")


[-2.15068568 -0.91108573]
if a performer spends on average a 200 minutes online in a show and has on average 3000 viewers, their average ranking will be:[719.31261827]


### Front pages
Collecting the column names from homepages front files (examples for each platform) and putting them all in one file for comparison

In [100]:
#getting a list of all the files in the front pages folder (here, only example files)
fronts_folder = "/Users/emilija/Desktop/homepages/fronts"
dir_l = os.listdir(fronts_folder)
print(dir_l)

#choosing a random file and printing it's name 
test_file = random.choice(dir_l)
print(test_file)

#reading the chosen file into a pandas dataframe and opening the output
test_df = pd.read_csv(f"{fronts_folder}/{test_file}")
test_df


['bongacams_2022-01-16_23-30-01_1531rooms.csv', 'myfreecams2022-01-16_23-30-01_843rooms.csv', 'livejasmin_2022-01-16_23-30-01_251rooms.csv', 'ebonycams_2022-01-16_23-30-01_9504rooms.csv', 'streamate_2022-01-16_23-30-01_1384rooms.csv', 'chaturbate_2022-01-16_23-30-01_4951rooms.csv']
ebonycams_2022-01-16_23-30-01_9504rooms.csv


,position,page,name,url,age,rating_in_full_stars_1_to_5,goldshow,offline,thumbnail_image_url,profile_picture_url,scrapetime
0,1,1,TinaGlamLove,https://ebonycams.com/cam/TinaGlamLove?langcho...,35,5.0,False,False,https://object-cdn.icfsys.com/smconnect-snapsh...,NaN,2022-01-16_23-30-47
1,2,1,DannyGirl,https://ebonycams.com/cam/DannyGirl?langchoice=en,60,5.0,False,False,https://object-cdn.icfsys.com/smconnect-snapsh...,NaN,2022-01-16_23-30-47
2,3,1,CubanaDoll,https://ebonycams.com/cam/CubanaDoll?langchoic...,21,5.0,False,False,https://object-cdn.icfsys.com/smconnect-snapsh...,NaN,2022-01-16_23-30-47
3,4,1,EbonyGoddessKira,https://ebonycams.com/cam/EbonyGoddessKira?lan...,23,5.0,False,False,https://object-cdn.icfsys.com/smconnect-snapsh...,NaN,2022-01-16_23-30-47
4,5,1,QueenAnitaDix,https://ebonycams.com/cam/QueenAnitaDix?langch...,47,5.0,False,False,https://object-cdn.icfsys.com/smconnect-snapsh...,NaN,2022-01-16_23-30-47
...,...,...,...,...,...,...,...,...,...,...,...
9499,9500,99,AbbyAndChris,https://ebonycams.com/cam/AbbyAndChris?langcho...,19,NaN,False,True,NaN,https://cdn.nsimg.net/biopic/320x240/99844353,2022-01-16_23-41-21
9500,9501,99,Ashleyjones18,https://ebonycams.com/cam/Ashleyjones18?langch...,22,NaN,False,True,NaN,https://cdn.nsimg.net/biopic/320x240/97908191,2022-01-16_23-41-21
9501,9502,99,SlaveNana,https://ebonycams.com/cam/SlaveNana?langchoice=en,53,NaN,False,True,NaN,https://cdn.nsimg.net/biopic/320x240/23893731,2022-01-16_23-41-21
9502,9503,99,valenshow720,https://ebonycams.com/cam/valenshow720?langcho...,26,NaN,False,True,NaN,https://cdn.nsimg.net/biopic/320x240/88820360,2022-01-16_23-41-21


In [101]:
#reading the files in the fronts folder and taking their column names, adding them all to one dataframe for comparison

columns_full = []
headers_l = []

for file in dir_l:
    file_dir = f"{fronts_folder}/{file}"
    df = pd.read_csv(file_dir)
    columns_l = df.columns.tolist()
    print(f"{file} has these columns: {columns_l}")
    columns_full.append(columns_l)
    headers_l.append(file)

columns_df = pd.DataFrame(columns_full).transpose()
columns_df.columns = headers_l
#columns_df = columns_df.drop("performerlist_sample_1000.csv", 1)
columns_df.to_csv("fronts_data_overview.csv")
columns_df

bongacams_2022-01-16_23-30-01_1531rooms.csv has these columns: ['position', 'page', 'name', 'subject', 'url', 'tags', 'vibrator', 'HD', 'HD+', 'mobile', 'new', 'private', 'group', 'away', 'lifted_up_webcam_model', 'thumbnail_image_url', 'social_media', 'scrapetime']
myfreecams2022-01-16_23-30-01_843rooms.csv has these columns: ['position', 'page', 'name', 'room_count', 'topic', 'thumbnail_image_url', 'group_show', 'private_show', 'true_private_show', 'club_show', 'preview_unavailable', 'new_model', 'golden_crown', 'silver_crown']
livejasmin_2022-01-16_23-30-01_251rooms.csv has these columns: ['position', 'page', 'name', 'url', 'VIP show', 'birthday_show', 'promoted', 'new', 'vibrator', 'video_call', 'mobile', 'profile_picture_url', 'scrapetime']
ebonycams_2022-01-16_23-30-01_9504rooms.csv has these columns: ['position', 'page', 'name', 'url', 'age', 'rating_in_full_stars_1_to_5', 'goldshow', 'offline', 'thumbnail_image_url', 'profile_picture_url', 'scrapetime']
streamate_2022-01-16_23-

,bongacams_2022-01-16_23-30-01_1531rooms.csv,myfreecams2022-01-16_23-30-01_843rooms.csv,livejasmin_2022-01-16_23-30-01_251rooms.csv,ebonycams_2022-01-16_23-30-01_9504rooms.csv,streamate_2022-01-16_23-30-01_1384rooms.csv,chaturbate_2022-01-16_23-30-01_4951rooms.csv
0,position,position,position,position,position,position
1,page,page,page,page,page,page
2,name,name,name,name,name,name
3,subject,room_count,url,url,url,subject
4,url,topic,VIP show,age,age,url
5,tags,thumbnail_image_url,birthday_show,rating_in_full_stars_1_to_5,rating_in_full_stars_1_to_5,tags
6,vibrator,group_show,promoted,goldshow,goldshow,location
7,HD,private_show,new,offline,thumbnail_image_url,viewers
8,HD+,true_private_show,vibrator,thumbnail_image_url,scrapetime,time
9,mobile,club_show,video_call,profile_picture_url,None,female
